In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
import numpy as np
import tensorflow as tf
import json

In [3]:
with open('dataset-bpe.json') as fopen:
    data = json.load(fopen)

In [4]:
train_X = data['train_X']
train_Y = data['train_Y']
test_X = data['test_X']
test_Y = data['test_Y']

In [5]:
EOS = 2
GO = 1
vocab_size = 32000

In [6]:
train_Y = [i + [2] for i in train_Y]
test_Y = [i + [2] for i in test_Y]

In [20]:
from tensorflow.python.util import nest
from tensorflow.python.layers.core import Dense

def gnmt_residual_fn(inputs, outputs):
    def split_input(inp, out):
        out_dim = out.get_shape().as_list()[-1]
        inp_dim = inp.get_shape().as_list()[-1]
        return tf.split(inp, [out_dim, inp_dim - out_dim], axis=-1)
    actual_inputs, _ = nest.map_structure(split_input, inputs, outputs)

    def assert_shape_match(inp, out):
        inp.get_shape().assert_is_compatible_with(out.get_shape())
    nest.assert_same_structure(actual_inputs, outputs)
    nest.map_structure(assert_shape_match, actual_inputs, outputs)
    return nest.map_structure(lambda inp, out: inp + out, actual_inputs, outputs)

class GNMTAttentionMultiCell(tf.nn.rnn_cell.MultiRNNCell):

    def __init__(self, attention_cell, cells, use_new_attention=True):
        cells = [attention_cell] + cells
        self.use_new_attention = use_new_attention
        super(GNMTAttentionMultiCell, self).__init__(
            cells, state_is_tuple=True)

    def __call__(self, inputs, state, scope=None):
        """Run the cell with bottom layer's attention copied to all upper layers."""
        if not nest.is_sequence(state):
            raise ValueError(
                "Expected state to be a tuple of length %d, but received: %s"
                % (len(self.state_size), state))

        with tf.variable_scope(scope or "multi_rnn_cell"):
            new_states = []

            with tf.variable_scope("cell_0_attention"):
                attention_cell = self._cells[0]
                attention_state = state[0]
                cur_inp, new_attention_state = attention_cell(
                    inputs, attention_state)
                new_states.append(new_attention_state)

            for i in range(1, len(self._cells)):
                with tf.variable_scope("cell_%d" % i):
                    cell = self._cells[i]
                    cur_state = state[i]

                    if self.use_new_attention:
                        cur_inp = tf.concat(
                            [cur_inp, new_attention_state.attention], -1)
                    else:
                        cur_inp = tf.concat(
                            [cur_inp, attention_state.attention], -1)

                    cur_inp, new_state = cell(cur_inp, cur_state)
                    new_states.append(new_state)
        return cur_inp, tuple(new_states)

class Translator:
    def __init__(self, size_layer, num_layers, embedded_size, learning_rate, beam_width = 5):
        
        def cells(size_layer=size_layer,reuse=False):
            return tf.nn.rnn_cell.LSTMCell(size_layer,initializer=tf.orthogonal_initializer(),reuse=reuse)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype=tf.int32)
        batch_size = tf.shape(self.X)[0]
        
        embeddings = tf.Variable(tf.random_uniform([vocab_size, embedded_size], -1, 1))
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        decoder_embedded = tf.nn.embedding_lookup(embeddings, decoder_input)
        
        num_residual_layer = num_layers - 2
        num_bi_layer = 1
        num_ui_layer = num_layers - num_bi_layer
        
        encoder_outputs, encoder_state = tf.nn.dynamic_rnn(
            cell = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)]), 
            inputs = tf.nn.embedding_lookup(embeddings, self.X),
            sequence_length = self.X_seq_len,
            dtype = tf.float32)
        
        decoder_cells = []
        for n in range(num_layers):
            cell = cells(size_layer)
            if (n >= num_layers - num_residual_layer):
                cell = tf.nn.rnn_cell.ResidualWrapper(cell, residual_fn = gnmt_residual_fn)
            decoder_cells.append(cell)
        attention_cell = decoder_cells.pop(0)
        to_dense = tf.layers.Dense(vocab_size)
        
        with tf.variable_scope('decode'):
            attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                num_units = size_layer, 
                memory = encoder_outputs,
                memory_sequence_length = self.X_seq_len)
            att_cell = tf.contrib.seq2seq.AttentionWrapper(
                cell = attention_cell,
                attention_mechanism = attention_mechanism,
                attention_layer_size = None,
                alignment_history = True,
                output_attention = False)
            gcell = GNMTAttentionMultiCell(att_cell, decoder_cells)
            
            self.initial_state = tuple(
                zs.clone(cell_state=es)
                if isinstance(zs, tf.contrib.seq2seq.AttentionWrapperState) else es
                for zs, es in zip(
                    gcell.zero_state(batch_size, dtype=tf.float32), encoder_state))
            
            training_helper = tf.contrib.seq2seq.TrainingHelper(
                decoder_embedded,
                self.Y_seq_len,
                time_major = False
            )
            training_decoder = tf.contrib.seq2seq.BasicDecoder(
                cell = gcell,
                helper = training_helper,
                initial_state = self.initial_state,
                output_layer = to_dense)
            training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = training_decoder,
                impute_finished = True,
                maximum_iterations = tf.reduce_max(self.Y_seq_len))
            self.training_logits = training_decoder_output.rnn_output
            
        with tf.variable_scope('decode', reuse=True):
            encoder_out_tiled = tf.contrib.seq2seq.tile_batch(encoder_outputs, beam_width)
            encoder_state_tiled = tf.contrib.seq2seq.tile_batch(encoder_state, beam_width)
            X_seq_len_tiled = tf.contrib.seq2seq.tile_batch(self.X_seq_len, beam_width)
            
            attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                num_units = size_layer, 
                memory = encoder_out_tiled,
                memory_sequence_length = X_seq_len_tiled)
            att_cell = tf.contrib.seq2seq.AttentionWrapper(
                cell = attention_cell,
                attention_mechanism = attention_mechanism,
                attention_layer_size = None,
                alignment_history = False,
                output_attention = False)
            gcell = GNMTAttentionMultiCell(att_cell, decoder_cells)
            
            self.initial_state = tuple(
                zs.clone(cell_state=es)
                if isinstance(zs, tf.contrib.seq2seq.AttentionWrapperState) else es
                for zs, es in zip(
                    gcell.zero_state(batch_size * beam_width, dtype=tf.float32), encoder_state_tiled))
            
            predicting_decoder = tf.contrib.seq2seq.BeamSearchDecoder(
                cell = gcell,
                embedding = embeddings,
                start_tokens = tf.tile(tf.constant([GO], dtype=tf.int32), [batch_size]),
                end_token = EOS,
                initial_state = self.initial_state,
                beam_width = beam_width,
                output_layer = to_dense,
                length_penalty_weight = 0.0)
            predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = predicting_decoder,
                impute_finished = False,
                maximum_iterations = 2 * tf.reduce_max(self.X_seq_len))
            self.fast_result = predicting_decoder_output.predicted_ids[:, :, 0]
            
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [18]:
size_layer = 512
num_layers = 3
learning_rate = 1e-3
batch_size = 64
epoch = 20

In [21]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Translator(size_layer, num_layers, size_layer, learning_rate)
sess.run(tf.global_variables_initializer())

/home/husein/.local/lib/python3.6/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [22]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [23]:
batch_x = pad_sequences(train_X[:10], padding='post')
batch_y = pad_sequences(train_Y[:10], padding='post')

sess.run([model.fast_result, model.cost, model.accuracy], 
         feed_dict = {model.X: batch_x, model.Y: batch_y})

[array([[25165, 25165, 25165, 12719, 12719, 12719, 12719, 11384,  4722,
          4722,  4722, 20577, 20577, 20577, 20577, 20577, 31007, 31007,
         31007, 31007, 31007, 31007, 31007, 31007, 31007, 31007,    95,
            95,    95,    95, 10788, 10788, 26129, 25836, 25836, 25836,
         25836, 25836, 25836, 28841, 22661,  6972,  6972,  6972,  6972,
          6972,  6972, 18721, 17965, 17965, 17965, 17965, 17965, 20057,
         20057,  6549,  6549,  4051,  4051,  4051, 27265, 25803, 25803,
         22130,  3280,  3280,  3280, 21261, 21261, 21261, 12184, 12184],
        [23674, 22063, 20393, 20393, 25949, 25949, 25949, 25949, 25949,
         24464, 24464, 24464, 24464, 24464, 24464,  9336,  9336, 13471,
         13471, 13471, 21559, 15741, 30712, 30712, 30712,  3493,  3493,
          3493,  3493,  3493,  7100,  7100,  7100,  7100, 24171,  7100,
         24171, 24171, 31383, 31383, 31383, 31383, 31383, 31383, 24730,
          1435,  1435,  1435,  1435,  1435, 24100, 24100, 17077

In [24]:
import tqdm

for e in range(epoch):
    pbar = tqdm.tqdm(
        range(0, len(train_X), batch_size), desc = 'minibatch loop')
    train_loss, train_acc, test_loss, test_acc = [], [], [], []
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        batch_x = pad_sequences(train_X[i : index], padding='post')
        batch_y = pad_sequences(train_Y[i : index], padding='post')
        feed = {model.X: batch_x,
                model.Y: batch_y}
        accuracy, loss, _ = sess.run([model.accuracy,model.cost,model.optimizer],
                                    feed_dict = feed)
        train_loss.append(loss)
        train_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    
    pbar = tqdm.tqdm(
        range(0, len(test_X), batch_size), desc = 'minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_X))
        batch_x = pad_sequences(test_X[i : index], padding='post')
        batch_y = pad_sequences(test_Y[i : index], padding='post')
        feed = {model.X: batch_x,
                model.Y: batch_y,}
        accuracy, loss = sess.run([model.accuracy,model.cost],
                                    feed_dict = feed)

        test_loss.append(loss)
        test_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    print('epoch %d, training avg loss %f, training avg acc %f'%(e+1,
                                                                 np.mean(train_loss),np.mean(train_acc)))
    print('epoch %d, testing avg loss %f, testing avg acc %f'%(e+1,
                                                              np.mean(test_loss),np.mean(test_acc)))

minibatch loop:   0%|          | 0/3125 [00:00<?, ?it/s]

epoch 1, training avg loss 5.345767, training avg acc 0.196012
epoch 1, testing avg loss 4.459814, testing avg acc 0.263600


minibatch loop:   0%|          | 0/3125 [00:00<?, ?it/s]

epoch 2, training avg loss 4.084258, training avg acc 0.297619
epoch 2, testing avg loss 3.909624, testing avg acc 0.319685


minibatch loop:   0%|          | 0/3125 [00:00<?, ?it/s]

...

epoch 18, training avg loss 1.182478, training avg acc 0.702417
epoch 18, testing avg loss 4.944494, testing avg acc 0.330336


minibatch loop:   0%|          | 0/3125 [00:00<?, ?it/s]

epoch 19, training avg loss 1.132978, training avg acc 0.711609
epoch 19, testing avg loss 5.042641, testing avg acc 0.330751


minibatch loop: 100%|██████████| 79/79 [00:14<00:00,  5.59it/s, accuracy=0.382, cost=4.28]

epoch 20, training avg loss 1.088114, training avg acc 0.719840
epoch 20, testing avg loss 5.144392, testing avg acc 0.328790


In [25]:
from tensor2tensor.utils import bleu_hook

In [26]:
results = []
for i in tqdm.tqdm(range(0, len(test_X), batch_size)):
    index = min(i + batch_size, len(test_X))
    batch_x = pad_sequences(test_X[i : index], padding='post')
    feed = {model.X: batch_x}
    p = sess.run(model.fast_result,feed_dict = feed)
    result = []
    for row in p:
        result.append([i for i in row if i > 3])
    results.extend(result)

100%|██████████| 79/79 [00:34<00:00,  2.30it/s]


In [27]:
rights = []
for r in test_Y:
    rights.append([i for i in r if i > 3])

In [28]:
bleu_hook.compute_bleu(reference_corpus = rights,
                       translation_corpus = results)

0.055380445